# 1. Interactive Quantum System Identifiaction Applet
## 1.1 Background
The following notebook demonstrates the current version of quantum system identification I've been working on. This notebook demonstrates the algorithm's ability to identify a random quantum system from simulated data.
- Select the order of the system
- Generate an appropriate basis
- Generate a random quantum state transition matrix
- Simulate data
- Add noise
- Regress data onto basis as $\dot{x}_{data}=\Theta w$
- Estimate original matrix

# 2. Import Functions
Mostly ipywidgets, bqplot, and numpy

In [2]:
import numpy as np
import pandas as pd
import math
from bqplot import pyplot as plt
import random
from IPython.display import display
from ipywidgets.widgets import IntSlider, HBox, Button, interact
import ipywidgets.widgets as widgets
from scipy import signal
%matplotlib inline


# 3. Select Sytem Order
Should work for any order, as long as you have RAM for the basis. Mostly tested with 2 through 5.
## n=

In [3]:
n=widgets.IntText(
    value=3,
    disabled=False
)
display(n)

IntText(value=3)

# 4. Generate a Basis for the Space
Not pauli like, but spans the space.

First, generates matricies with a single element in one of the diagonals.

Then, generate diagonal element for the real component, ie (1,3 & 3,1) with the same sign, for Hamiltonian structure.

Finally, generate diagonal element pairs for imaginary component with opposite sign.

In [4]:
basis_button = widgets.Button(description="Generate Basis")
output = widgets.Output()
#display(basis_button, output)
def on_basis_button_clicked(b):
    #diagonals
    output.clear_output()
    diag=np.zeros((n.value,n.value,n.value),dtype=complex);
    for i in range(n.value):
        diag[i,i,i]=0+1j
        
    idx_start=1;
    counter=-1;
    count=int(((n.value*n.value)-n.value)/2)
    off_complex=np.zeros((count,n.value,n.value),dtype=complex);
    for i in range(0,n.value-1):
        for ii in range(idx_start,n.value):
            if i != ii and i<ii:
                off_complex[counter,i,ii]=0+1j
                off_complex[counter,ii,i]=0+1j
                counter=counter+1
                
    idx_start=1;
    counter=-1;
    count=int(((n.value*n.value)-n.value)/2)
    off_real=np.zeros((count,n.value,n.value),dtype=complex);
    for i in range(0,n.value-1):
        for ii in range(idx_start,n.value):
            if i != ii and i<ii:
                off_real[counter,i,ii]=1+0j
                off_real[counter,ii,i]=-1+0j
                counter=counter+1

    basis=np.concatenate((diag,off_complex,off_real));
    basis_button.value=basis
    with output:
        display(basis)




basis_button.on_click(on_basis_button_clicked)

# 5. Generate a Random Matrix
## 5.1 Matrix Options
First, select from a set of options. Default will generate a matrix with random entries in every possible element of the state transition matrix. Off diagonal only leaves zeros on the diagonal. 

In [5]:
option=widgets.RadioButtons(
    options=['default','Off Diagonal Matrix Only?', 'Upper Diagonal Only?', 'Lower Diagonal Only?'],
#     value='pineapple',
    description='Settings:',
    disabled=False
)
display(option)

RadioButtons(description='Settings:', options=('default', 'Off Diagonal Matrix Only?', 'Upper Diagonal Only?',…

## 5.2 Generate Random State Matrix
This button can be clicked repeatedly. Adds a random component of each basis to form a state matrix for identification.

In [6]:
matrix_button = widgets.Button(description="Generate Matrix")
output = widgets.Output()
#display(matrix_button, output)
def on_matrix_button_clicked(b):
    output.clear_output()
    
    if option.value == 'default':
        A=np.zeros((n.value,n.value),dtype=complex)
        for i in range(len(basis_button.value)):
            A=A+np.squeeze((basis_button.value[i]*random.random()))
            
    if option.value == 'Upper Diagonal Only?':
        A=np.zeros((n.value,n.value),dtype=complex)
        for i in range(0,1):
            A=A+np.squeeze((basis_button.value[i]*random.random()))
            
    if option.value == 'Lower Diagonal Only?':
        A=np.zeros((n.value,n.value),dtype=complex)
        A=A+np.squeeze((basis_button.value[n.value-1]*random.random()))
        
    if option.value == 'Off Diagonal Matrix Only?':
        A=np.zeros((n.value,n.value),dtype=complex)
        for i in range((len(basis_button.value)-n.value),len(basis_button.value)):
            A=A+np.squeeze((basis_button.value[i]*random.random()))
            
    matrix_button.value=A
    
    

    with output:
        display(A)



matrix_button.on_click(on_matrix_button_clicked)

# 6. Generate Data
Form the system in state space terms, and use ODE45 to generate 1000 points of data over 10 seconds. Plot the timeseries data.

In [7]:
#matrix_button.value=np.array([[0-1j, 0],[0,0]])

In [8]:
simdata_button = widgets.Button(description="Simulate Data")

t = np.linspace(0, 10, 1000, endpoint=False)
output = widgets.Output()
#display(simdata_button, output)
def on_simdata_button_clicked(b):
    output.clear_output()
    Xinit=np.ones(n.value)-(1j*np.ones(n.value))

    sys = signal.StateSpace(matrix_button.value, np.zeros((n.value,1)), np.zeros((1,n.value)),0)
    

    tout, yout, xout = signal.lsim(sys,U=0,T=t,X0=Xinit)
    plt.clear()
    
    fig = plt.figure(title="Simulated Data")
    line_chart = plt.plot(x=tout, y=np.real(np.transpose(xout)))
    line_chart1 = plt.plot(x=tout, y=np.imag(np.transpose(xout)))

    simdata_button.value=xout
    

    
    

    with output:
        display(fig)




simdata_button.on_click(on_simdata_button_clicked)

Button(description='Simulate Data', style=ButtonStyle())

Output()

# 7. Regress Data Onto Basis Library Using Only Output Data
In SINDY like fashion, pass data through each basis function to generate $\Theta$. Regress onto the numerical derivative of the data, $\dot{x}$, as $\dot{x}=\Theta w$. Result, $w$ is the coefficient in front of each basis. 

In [9]:
regress_button = widgets.Button(description="Estimate State Matrix")
output = widgets.Output()
#display(regress_button, output)
def on_regress_button_clicked(b):
    output.clear_output()
    Theta_cust=np.empty((len(simdata_button.value), 0))
    for i in range(len(basis_button.value)):
        x_sigma=np.matmul(simdata_button.value,basis_button.value[i])
        Theta_cust=np.append(Theta_cust,x_sigma,axis=1)
    Theta_cust=np.reshape(Theta_cust,[-1,len(basis_button.value)],'F')
    Theta=np.concatenate((np.real(Theta_cust),np.imag(Theta_cust)),axis=0) 

    dx=np.gradient(simdata_button.value,0.01,axis=0)
    dx2=np.squeeze(np.concatenate((np.real(dx.flatten('F')),np.imag(dx.flatten('F')))))
    Xi=np.linalg.lstsq(Theta,dx2,rcond=None)
    
    lambda_knob=0.1
    smallinds=np.argwhere(abs(Xi[0])<lambda_knob)
    Theta_small=np.delete(Theta,smallinds,axis=1);
    basis_small=np.delete(basis_button.value,smallinds,axis=0)
    Xi=np.linalg.lstsq(Theta_small,dx2,rcond=None)
    A_solve=np.zeros([n.value,n.value])
    for i in range(len(basis_small)):
        if np.all(np.isreal(basis_small[i])):
            Xi[0][i]=-Xi[0][i]
        A_solve=A_solve+(Xi[0][i]*basis_small[i])
        
    regress_button.value=A_solve
        

    
    

    with output:
        display(A_solve)




regress_button.on_click(on_regress_button_clicked)

In [10]:
# Xi=regress_button.value; 
# basis=basis_button.value; basis
# for i in range(len(basis)):
#     if np.all(np.isreal(basis[i])):
#         Xi[0][i]=-Xi[0][i]
    

# 8. Compare Results & Demo

In [11]:
results_button = widgets.Button(description="Compare Results")
output = widgets.Output()

display(basis_button,matrix_button,simdata_button,regress_button,results_button, output)
# display(matrix_button.value)
# display(regress_button.value)


      
def on_results_button_clicked(b):
    output.clear_output()
    A_solve=regress_button.value
    Xinit=np.ones(n.value)-(1j*np.ones(n.value))
    norm_error=np.linalg.norm((matrix_button.value-regress_button.value))
    sys_org = signal.StateSpace(matrix_button.value, np.zeros((n.value,1)), np.zeros((1,n.value)),0)
    sys_id = signal.StateSpace(regress_button.value, np.zeros((n.value,1)), np.zeros((1,n.value)),0)
    tout, yout, xout = signal.lsim(sys_org,U=0,T=t,X0=Xinit)
    tout_id, yout_id, xout_id = signal.lsim(sys_id,U=0,T=t,X0=Xinit)
    plt.clear()
    result=[]
    result=np.concatenate((np.real(np.transpose(xout)),np.imag(np.transpose(xout)),np.real(np.transpose(xout_id)),np.imag(np.transpose(xout_id))))
    result_sim=np.concatenate((np.real(np.transpose(xout)),np.imag(np.transpose(xout))))
    result_id=np.concatenate((np.real(np.transpose(xout_id)),np.imag(np.transpose(xout_id))))
    fig = plt.figure(title="Compare Simulated Data to Identified System")
    line_chart =plt.plot(tout, result_id, 'ks--')   
    line_chart =plt.plot(x=tout, y=result_sim) 
    
    
    with output:
        display(fig,A_solve,matrix_button.value,['Norm Error= '+ str(norm_error)])




results_button.on_click(on_results_button_clicked)

Button(description='Generate Basis', style=ButtonStyle())

Button(description='Generate Matrix', style=ButtonStyle())

Button(description='Simulate Data', style=ButtonStyle())

Button(description='Estimate State Matrix', style=ButtonStyle())

Button(description='Compare Results', style=ButtonStyle())

Output()